## 한식진흥원 레시피 데이터를 이용한 추천 시스템
- 재료 데이터를 형태소 분석을 통해 명사만 추출하여 분석 진행
#### 형태소 분석 : Kiwi

In [426]:
import pandas as pd
import MeCab
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
STOP_WORDS = []
#['두께','절단','정도','약간','컵','큰술','작은술','술','다리','길이','것','체','분','간','모','가','등','줄','토막','가루',
#'마리','손질','후','적당량','개','쪽','살','장','잎','대','즙','단','선택','사항','불린','공기','당량','건조','습식','제거']
mecab = MeCab.Tagger()

In [427]:
def extract_nouns(text):
    mecab = MeCab.Tagger()
    nouns = []
    node = mecab.parseToNode(text)
    while node:
        # 품사 정보는 node.feature에 저장됨
        # 첫 번째 항목이 품사 대분류
        pos = node.feature.split(',')[0]
        # 명사 관련 품사 태그들
        if pos in ['NNG', 'NNP', 'NNB']:  # 일반명사, 고유명사, 의존명사
            if node.surface and (node.surface not in STOP_WORDS):  # 빈 문자열이 아닌 경우만
                nouns.append(node.surface)
        node = node.next
    return ' '.join(nouns)

In [428]:
## 데이터셋
data_raw = pd.read_csv('./dataset/추천용데이터셋.csv')
data = data_raw.copy()
## 재료 리스트 형태소 분석
data['코드'] = data['코드'].astype('int32')
data['재료태그'] = data['재료'].apply(extract_nouns)
data.head()

,대분류,소분류,메뉴명,코드,분량,재료,재료태그
0,구이,갈비구이,소갈비구이,56920,4인분,갈비(두께 1cm 정도로 뼈째 두툼하게 절단) 1kg 간장 5큰술 설탕 2큰술 ...,갈비 두께 정도 뼈 절단 간장 술 설탕 술 파 술 마늘 술 깨소금 술 참기름 술 후...
1,구이,고등어구이,고등어구이,56908,4인분,고등어 2마리(손질 후 500g) 소금 2큰술 식용유 2큰술,고등어 손질 후 소금 술 식용유 술
2,구이,곱창구이,곱창구이,56909,4인분,돼지(또는 소) 곱창 1kg 밀가루 1컵 굵은소금 1큰술 식용유 적당량 감자...,돼지 소 곱창 밀가루 컵 굵은소금 술 식용유 당량 감자 마늘 부추 생채 부추 양파 ...
3,구이,닭갈비,닭갈비,56916,4인분,닭다리 살 480g 떡볶이 떡 120g 고구마 90g 양배추 잎 4장 당근 ...,닭다리 떡볶이 떡 고구마 양배추 잎 장 당근 마늘 양파 개 대파 식용유 술 고추장 ...
4,구이,더덕구이,더덕구이,56926,4인분,더덕 480g 통깨 1큰술 식용유 ½컵 간장 1큰술 참기름 2큰술 간장 1큰술...,더덕 통깨 술 식용유 컵 간장 술 참기름 술 간장 술 고추장 술 고춧가루 술 설탕 ...


In [429]:
## 벡터화 대상 데이터 구성
target = data[['코드']].astype(str)
target['재료'] = data['재료태그']
target.head()

,코드,재료
0,56920,갈비 두께 정도 뼈 절단 간장 술 설탕 술 파 술 마늘 술 깨소금 술 참기름 술 후...
1,56908,고등어 손질 후 소금 술 식용유 술
2,56909,돼지 소 곱창 밀가루 컵 굵은소금 술 식용유 당량 감자 마늘 부추 생채 부추 양파 ...
3,56916,닭다리 떡볶이 떡 고구마 양배추 잎 장 당근 마늘 양파 개 대파 식용유 술 고추장 ...
4,56926,더덕 통깨 술 식용유 컵 간장 술 참기름 술 간장 술 고추장 술 고춧가루 술 설탕 ...


In [468]:
## 카운터 벡터라이즈
documents = target['재료'].tolist()
# vectorizer = CountVectorizer(
#     max_features=1000,      # 최대 특성 수
#     min_df=1,              # 최소 문서 빈도
#     max_df=0.8,            # 최대 문서 빈도 비율
#     ngram_range=(1, 2),    # n-gram 범위 (1-gram, 2-gram)
#     lowercase=True,        # 소문자 변환
# )
vectorizer = TfidfVectorizer(
    min_df=1,
    max_df=0.8,
    ngram_range=(1, 2),
    sublinear_tf=True,
    smooth_idf=True,         # IDF 스무딩
    norm='l2',               # 정규화 방법 ('l1', 'l2', None)
    use_idf=True             # IDF 사용 여부
)
vector_ = vectorizer.fit_transform(documents)
features = vectorizer.get_feature_names_out()
vector = vector_.toarray()
similarity = cosine_similarity(vector,vector)

In [471]:
## 유사한 음식 상위 2개
idx = 0
find_idx = np.argsort(similarity[idx])[::-1][:3]
print(data.iloc[idx,:])
data.iloc[find_idx]

대분류                                                    구이
소분류                                                  갈비구이
메뉴명                                                 소갈비구이
코드                                                  56920
분량                                                    4인분
재료      갈비(두께 1cm 정도로 뼈째 두툼하게 절단) 1kg 간장 5큰술  설탕 2큰술  ...
재료태그    갈비 두께 정도 뼈 절단 간장 술 설탕 술 파 술 마늘 술 깨소금 술 참기름 술 후...
Name: 0, dtype: object


,대분류,소분류,메뉴명,코드,분량,재료,재료태그
0,구이,갈비구이,소갈비구이,56920,4인분,갈비(두께 1cm 정도로 뼈째 두툼하게 절단) 1kg 간장 5큰술 설탕 2큰술 ...,갈비 두께 정도 뼈 절단 간장 술 설탕 술 파 술 마늘 술 깨소금 술 참기름 술 후...
68,회,육회,육회,56978,4인분,소고기(우둔살) 400g 배 ½개(300g) 마늘 2톨 잣가루 2큰술 간장 4...,소고기 우둔살 배 개 마늘 잣가루 술 간장 술 설탕 술 파 술 마늘 술 깨소금 술 ...
6,구이,불고기,불고기,56912,4인분,소고기(등심) 500g 양파 1개(200g) 표고버섯(또는 느타리버섯) 3개(90...,소고기 등심 양파 표고버섯 느타리버섯 실파 식용유 술 간장 술 설탕 술 파 술 마늘...


In [395]:
similarity[0,[0,68,6,75]]

array([1.        , 0.68853037, 0.68853037, 0.62554324])